# Coleções, geometrias e filtros: extraindo uma imagem dos datasets do GEE em uma determinada data e local

Apresentamos aqui a primeira consulta efetiva utilizando a plataforma do Google Earth Engine API. Lembrando que, antes de se realizar qualquer operação com o GEE, devemos iniciar a conexão, conforme o primeiro notebook. Entretanto, essa operação será feita apenas uma vez (até a validade do token).

Neste notebook, iremos consultar o GEE por uma coleção, em uma determinada área de estudo e aplicar um filtro datas.

Primeiramente, vamos importar a biblioteca e inicializá-la:

In [1]:
import ee # importação
ee.Initialize() # inicialização

Agora, vamos definir a geometria (baseada na Latitude e Longitude) da nossa área de estudo. Neste caso, faremos uma consulta ao Reservatório Hidrelétrico de Barra Bonita, localizado no interior do Estado de São Paulo (https://earth.google.com/web/@-22.57959722,-48.38199298,475.1139016a,35077.08122335d,35y,18.39831754h,34.08243439t,0r/data=Ck8aTRJHCiUweDk0Yzc0ZjdkZjg4YzFkMzc6MHg0Mzk3M2ViYmMyZTQ0Y2Y3GUg0gSIWeTbAIVy91BkpSEjAKgxCYXJyYSBCb25pdGEYASAB):

In [2]:
# Notem que foi criada uma coordenada (Latitude e Longitude) através de uma string, posteriormente repartida pelas virgulas
# Essa abordagem é importante para quando utilizarmos a linha da comando
coordenadas = "-48.52353991121163,-22.522557296253044,-48.298663507403035,-22.625905269537693"

# Aqui, usamos uma ferramenta do Python chamada de unpacking
x1,y1,x2,y2 = coordenadas.split(",")

# Criamos a geometria com base nas coordenadas 'quebradas' acima
geometria = geometry = ee.Geometry.Polygon(
        [[[float(x1),float(y2)],
          [float(x2),float(y2)],
          [float(x2),float(y1)],
          [float(x1),float(y1)],
          [float(x1),float(y2)]]])

# Podemos, também, extrair as coordenadas centrais da área de estudo
latitude_central = (float(x1)+float(x2))/2
longitude_central = (float(y1)+float(y2))/2

Da mesma forma que fizemos para as coordenadas, criaremos uma string contendo duas datas (inicial e final), de onde filtraremos a coleção do GEE:

In [3]:
# string de datas (poderia vir, por exemplo, da linha de comando como um argumento)
datas = "2014-10-13,2014-10-14"

# Divisão das duas datas pela vírgula, novamente usando a técnica de unpacking
inicio,fim = datas.split(",")

Neste momento, estamos prontos para consultar uma coleção do GEE. Os datasets disponíveis, assim como os sensores, podem ser consultados diretamente do site do Google Eearth Engine API (https://developers.google.com/earth-engine/datasets). Neste caso, utilizaremos o sensor Landsat-8/OLI (Dataset chamado de: LANDSAT/LC08/C01/T1_SR):

In [4]:
# Consultando a coleção com base na área de estudo e datas selecioandas
# Notem que utilizamos o filtro 'CLOUD_COVER'. Vários datasets possuem esses 'metadados' que podem ser utilizados para, neste caso, pegar as imagens com menos nuvem possível
colecao = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(geometria).filterDate(inicio,fim).filterMetadata('CLOUD_COVER','less_than', 30)

# Mostrar o total de imagens encontradas
print("Total de imagens encontradas: "+str(colecao.size().getInfo()))

Total de imagens encontradas: 1


Maiores informações sobre os metadados deste sensor Landsat-8/OLI: 

- https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR.

Uma informação importante é que muitos datasets possuem versões com correção atmosférica já aplicadal. Verfiquem antes de escolher! 

Outra dica importante é que a plataforma GEE possuí um ambiente desenvolvido em Javascript onde todos os códigos podem ser previamente testados. A sintaxe da linguagem é moderamente parecida com o Python, o que irá facilitar tradução do scripts:

- https://code.earthengine.google.com/

In [14]:
colecao = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(geometria)
imagem = colecao.median()
print(imagem.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa']


In [12]:
colecao = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(geometria)
imagem = colecao.median()
print(imagem.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60']


In [13]:
colecao = ee.ImageCollection('MODIS/006/MOD09GA').filterBounds(geometria)
imagem = colecao.median()
print(imagem.bandNames().getInfo())

['num_observations_1km', 'state_1km', 'SensorZenith', 'SensorAzimuth', 'Range', 'SolarZenith', 'SolarAzimuth', 'gflags', 'orbit_pnt', 'granule_pnt', 'num_observations_500m', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07', 'QC_500m', 'obscov_500m', 'iobs_res', 'q_scan']


In [36]:
from datetime import datetime as dt
from datetime import timedelta

# User input
a = dt.strptime("2020-10-01", "%Y-%m-%d")
b = dt.strptime("2020-10-08", "%Y-%m-%d")
print(a)
print(b)

2020-10-01 00:00:00
2020-10-08 00:00:00


In [37]:
# Time series
b_ = dt.now() if a > dt.now() else a
a_ = b_ - timedelta(days=365)
print(a_)
print(b_)

2019-04-02 16:27:06.018595
2020-04-01 16:27:06.018595
